In [4]:
from abc import abstractmethod

In [8]:
class Cart:
  def __init__(self):
    self.resturant=None
    self.cart_items=[]

  def get_sum(self):
    return sum(item.price for item in self.cart_items)

  def get_items(self):
    return self.cart_items

  def add_item(self,item):
    self.cart_items.append(item)


class User:
  def __init__(self,user_id,name,address):
    self.user_id=user_id
    self.usr_name=name.lower()
    self.usr_address=address.lower()
    self.cart=Cart()


class OrderManager:
    _instance = None

    def __init__(self):
        self.orders = []

    @staticmethod
    def get_instance():
        if not OrderManager._instance:
            OrderManager._instance = OrderManager()
        return OrderManager._instance

    def add_order(self, order):
        self.orders.append(order)

    def list_orders(self):
        print("\n--- All Orders ---")
        for order in self.orders:
            print(f"{order.get_type()} order for {order.user.name} | Total: ₹{order.total} | At: {order.scheduled}")

# RestaurantManager.py
class ResturantManager:
  _isInstance=None

  def __init__(self):
    self.resturants=[]

  @classmethod
  def get_instance(cls):
    if not cls._isInstance:
      cls._isInstance=ResturantManager()

    return cls._isInstance

  def add_resturant(self,res):
    self.resturants.append(res)

  def search_by_loc(self,loc):
    location=loc.lower()
    return [res for res in self.resturants if res.res_address==location]


class Resturant:
  next_id=1
  def __init__(self,name,address):
    self.res_id=Resturant.next_id
    Resturant.next_id+=1
    self.name=name.lower()
    self.res_address=address.lower()
    self.menu_items=[]

  def add_resturant(self,res):
    self.menu_items.append(res)

  def remove_resturant(self,id):
    self.menu_items.remove(menu_items[id])

  def get_items(self):
    return self.menu_items


class MenuItems:
  def __init__(self,id,name,price):
    self.name=name.lower()
    self.id=id
    self.price=price


class Order:
  id=1
  def __init__(self):
    self.order_id=id
    Order.id+=1
    self.user=None
    self.resturant=None
    self.total=0.0
    self.items=[]
    self.time=""
    self.stragey=None
    self.order_type=None


  def pay_now(self):
    if self.stragey:
      self.stragey.pay(self.total)
      return True
    else:
      print('Please select payment Stagey first')
      return False


  @abstractmethod
  def get_val(self):
    pass

class PickUpOrder(Order):

  def __init__(self):
    super().__init__()
    self.resturant_address=None

  def get_val(self):
    return "PICKUP"

class DileveryOrder(Order):

  def __init__(self):
    super().__init__()
    self.user_address=None

  def get_val(self):
    return "DILEVERY"

class OrderFactory:
  @abstractmethod
  def create_order(self,user,resturant,order_type,stragey):
    pass


class ScheduledOrderFactory(OrderFactory):

  def __init__(self,time):
    self.time=time

  def create_order(self,user,cart,resturant,order_type,stragey):

    if order_type=="DILEVERY":
      order=DileveryOrder()
      order.user_address=user.usr_address
    else:
      order=PickUpOrder()
      order.resturant_address=resturant.res_address

    order.user=user
    order.resturant=resturant
    order.order_type=order.get_val()
    order.stragey=stragey

    order.total=cart.get_total()
    order.items=cart.get_items()

    return order


class NowOrderFactory(OrderFactory):

  def create_order(self,user,cart,resturant,order_type,stragey):

    if order_type=="DILEVERY":
      order=DileveryOrder()
      order.user_address=user.usr_address
    else:
      order=PickUpOrder()
      order.resturant_address=resturant.res_address

    order.user=user
    order.resturant=resturant
    order.order_type=order.get_val()
    order.stragey=stragey
    order.time="current_time"
    order.total=cart.get_total()
    order.items=cart.get_items()

    return order


class PaymentStragey:
  @abstractmethod
  def pay(self,amount):
    pass

class UPIPaymentStragey(PaymentStragey):
  def __init__(self,upi_id):
    self.upi_id=upi_id

  def pay(self,amount):
    print(f"Amount Paid of Rs {amount} through UPI-ID {self.upi_id}")



class Tomato:

  def __init__(self):
    self.res_manager=ResturantManager.get_instance()
    self.ord_manager=OrderManager.get_instance()
    self.initilize()

  def initialize(self):
    # Create sample restaurants
    res1 = Resturant("Pizza Palace", "MG Road")
    res2 = Resturant("Burger Hub", "MG Road")
    res3 = Resturant("Sushi Corner", "Park Street")

    # Add menu items to restaurants
    res1.menu_items = [
        MenuItems(1, "Margherita Pizza", 250),
        MenuItems(2, "Farmhouse Pizza", 300)
    ]

    res2.menu_items = [
        MenuItems(3, "Veg Burger", 120),
        MenuItems(4, "Cheese Burger", 150)
    ]

    res3.menu_items = [
        MenuItems(5, "California Roll", 350),
        MenuItems(6, "Spicy Tuna Roll", 400)
    ]

    # Register restaurants with manager
    self.res_manager.add_resturant(res1)
    self.res_manager.add_resturant(res2)
    self.res_manager.add_resturant(res3)

    print("🍅 Tomato system initialized with sample restaurants and menus.")






  def search_resturant(self,loc):
    return self.res_manager.search_by_location(loc)

  def select_resturant(self,user,resturant):
    user.cart.resturant=resturant

  def add_to_cart(self,user,item_id):
    for item in user.cart.resturant:
      if item.id==item_id:
        print(f"Item with Item_id {item_id} added to the cart")
        user.cart.add_item(item)

  def checkoutnow(self,user,order_type,stragey):
    return self.checkout(user,order_type,stragey,NowOrderFactory())

  def checkoutscheduled(self,user,resturant,order_type,stragey):
    return self.checkout(user,order_type,stragey,ScheduledOrderFactory())

  def checkout(self,user,order_type,stragey,factory):
    if user.cart.is_empty:
      print("please select resturant first")
      return
    order=factory.create_order(user,user.cart,user.resturant,order_type,stragey)
    ord_manager.get_instance.add_order(order)
    return order

  def payment_now(self,user,order):
    if order.pay_now():

      user.cart.clear()








